In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from IPython.display import display, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import functions as funcs
import cell_class as cell


In [2]:
# Change output figure size
# ...needs to be in its own cell for some reason...

plt.rcParams['figure.figsize'] = [20, 5]

Jia, H., Rochefort, N. L., Chen, X., & Konnerth, A. (2011).
In vivo two-photon imaging of sensory-evoked dendritic calcium signals in cortical neurons.
Nature protocols, 6(1), 28.

Podgorski, K., & Haas, K. (2013).
Fast non‐negative temporal deconvolution for laser scanning microscopy.
Journal of biophotonics, 6(2), 153-162.

Coleman, P. (2019).
pyNeuroTrace. https://github.com/padster/pyNeuroTrace.git



To install pyneuortrace use this:
 pip install --upgrade "git+https://github.com/padster/pyNeuroTrace#egg=pyneurotrace&subdirectory=pyneurotrace"

## Global Settings

In [3]:
DATA_DIRECTORY = '/mnt/5404b8a5-71b7-4464-9a1e-b40cd26fac58/Data_Drive/Wissam/osfstorage-archive/Data (Images)/'

# Framerate of video in hz
HZ = 10

# Standard Deviation Threshold Multiplier
STD_THRESH = 5

# Power for Butterworth Lowpass filter
Power = 3

# Frequency cut-off
FC = 3

### Collects all the cells in the analysis data directory and groups them by condition in two lists

In [4]:
path = os.fspath(DATA_DIRECTORY)
cells = sorted((os.listdir(path)))
WT_Cells = []
YAC128_Cells = []
for folder in cells:
    if 'WT'  in folder:
        WT_Cells.append(folder[:-2])
    if 'YAC128'  in folder:
        YAC128_Cells.append(folder[:-2])
        
WT_Cells = np.unique(WT_Cells)
YAC128_Cells = np.unique(YAC128_Cells)


In [5]:
# Preform analysis

wt_results = pd.DataFrame(columns=['Name', 'Cell'])
for exp in tqdm(WT_Cells):
    wt_results = wt_results.append({'Name':exp,
                       'Cell': cell.cell(DATA_DIRECTORY, exp, HZ, Power, FC, STD_THRESH)}, ignore_index=True )
    

Cell 17_WT_Iono_180_S : Analysis Beginning
Cell 17_WT_Iono_180_S : Images Loaded
Cell 17_WT_Iono_180_S : Traces Completed
Cell 17_WT_Iono_180_S : Analysis Completed

Cell 18_WT_Iono_180_S : Analysis Beginning
Cell 18_WT_Iono_180_S : Images Loaded
Cell 18_WT_Iono_180_S : Traces Completed
Cell 18_WT_Iono_180_S : Analysis Completed




In [6]:
# Preform analysis

yac128_results = pd.DataFrame(columns=['Name', 'Cell'])
for exp in tqdm(YAC128_Cells):
    yac128_results = yac128_results.append({'Name':exp,
                       'Cell': cell.cell(DATA_DIRECTORY, exp, HZ, Power, FC, STD_THRESH)}, ignore_index=True )
    

Cell 14_YAC128_Iono_270_S : Analysis Beginning
Cell 14_YAC128_Iono_270_S : Images Loaded
Cell 14_YAC128_Iono_270_S : Traces Completed
Cell 14_YAC128_Iono_270_S : Analysis Completed

Cell 22_YAC128_Iono_180_D : Analysis Beginning
Cell 22_YAC128_Iono_180_D : Images Loaded
Cell 22_YAC128_Iono_180_D : Traces Completed
Cell 22_YAC128_Iono_180_D : Analysis Completed

Cell 29_YAC128_Iono_180_D : Analysis Beginning
Cell 29_YAC128_Iono_180_D : Images Loaded
Cell 29_YAC128_Iono_180_D : Traces Completed
Cell 29_YAC128_Iono_180_D : Analysis Completed

Cell 46_YAC128_Iono_060_S : Analysis Beginning
Cell 46_YAC128_Iono_060_S : Images Loaded
Cell 46_YAC128_Iono_060_S : Traces Completed
Cell 46_YAC128_Iono_060_S : Analysis Completed

Cell 47_YAC128_Iono_180_S : Analysis Beginning
Cell 47_YAC128_Iono_180_S : Images Loaded
Cell 47_YAC128_Iono_180_S : Traces Completed
Cell 47_YAC128_Iono_180_S : Analysis Completed

Cell 48_YAC128_Iono_088_S : Analysis Beginning
Cell 48_YAC128_Iono_088_S : Images Loaded
C

In [7]:
WT_Stats = pd.DataFrame()
WT_Shared_Peaks = pd.DataFrame()
WT_GCaMP = pd.DataFrame()
WT_RCaMP = pd.DataFrame()

WT_GTrace = pd.DataFrame()
WT_RTrace = pd.DataFrame()

WT_GDFF = []
WT_RDFF = []

WT_GDFF_index = []
WT_RDFF_index = []

for i in range(wt_results.shape[0]):
    WT_GDFF_index.append(wt_results.iloc[i,1].cell)
    WT_RDFF_index.append(wt_results.iloc[i,1].cell)
    
    WT_RDFF.append(pd.DataFrame(wt_results.iloc[i,1].nndR))
    WT_GDFF.append(pd.DataFrame(wt_results.iloc[i,1].nndG))
        
    WT_Stats = WT_Stats.append(wt_results.iloc[i,1].cell_stats, ignore_index=True)
    WT_Shared_Peaks = WT_Shared_Peaks.append(wt_results.iloc[i,1].shared_peak_data, ignore_index=True)
    WT_GCaMP = WT_GCaMP.append(wt_results.iloc[i,1].gcamp_peak_data, ignore_index=True)
    WT_RCaMP = WT_RCaMP.append(wt_results.iloc[i,1].rcamp_peak_data, ignore_index=True)

WT_GDFF = pd.concat(WT_GDFF, axis=1) 
WT_GDFF = WT_GDFF.transpose()
WT_GDFF.insert(loc=0, column='Cell', value=WT_GDFF_index)
WT_RDFF = pd.concat(WT_RDFF, axis=1)   
WT_RDFF = WT_RDFF.transpose()
WT_RDFF.insert(loc=0, column='Cell', value=WT_RDFF_index)

## Save Results for WT to CSV

In [8]:
np.set_printoptions(threshold=8000)
WT_Stats.to_csv("WT_Stats.csv", index=False)
WT_Shared_Peaks.to_csv("WT_Shared_Peaks.csv", index=False)
WT_GCaMP.to_csv("WT_GCaMP.csv", index=False)
WT_RCaMP.to_csv("WT_RCaMP.csv", index=False)

WT_RTrace.to_csv("WT_R_Trace.csv", index=False)
WT_GTrace.to_csv("WT_G_Trace.csv", index=False)
WT_RDFF.to_csv("WT_R_DFF.csv", index=False)
WT_GDFF.to_csv("WT_G_DFF.csv", index=False)

np.set_printoptions(threshold=10)

## Display Results for WT

In [9]:
display(WT_Stats)
display(WT_Shared_Peaks)
display(WT_GCaMP)
display(WT_RCaMP)

,Cell ID,GCaMP Peaks,RCaMP Peaks,Shared Peaks,GCaMP Percent Shared,RCaMP Percent Shared,Experiment Length (s),Drug Application,Iono GCaMP Max,Iono GCaMP Min,Iono GCaMP Dif,Iono RCaMP Max,Iono RCaMP Min,Iono RCaMP Dif,Prominence Ratio Mean,Prominence Ratio SEM
0,Cell 17_WT_Iono_180_S,18,20,18,1.0,0.900000,257.2,180.0,30.786286,6.418518,24.367769,19.248580,14.409737,4.838843,6.777985,1.028043
1,Cell 18_WT_Iono_180_S,8,14,8,1.0,0.571429,252.9,180.0,29.194602,7.769499,21.425103,18.144112,10.430527,7.713585,0.900000,0.294361


,Cell ID,GCaMP Area,GCaMP Loc (Frame),GCaMP Prominence,GCaMP Start,GCaMP Width,Peak Time Diff (G-R),Promicence Ratio (G/R),RCaMP Area,RCaMP Loc (Frame),RCaMP Prominence,RCaMP Start,RCaMP Width,Start Difference (G-R) Seconds,Start Difference 10% Max(G-R) Seconds,GCaMP Loc,RCaMP Loc,Start Difference (G-R)
0,Cell 17_WT_Iono_180_S,7.533122,114.0,0.500777,141.984245,2.650021,0.1,1.675790,4.153370,113.0,0.301117,151.286947,2.654281,-0.093027,-0.166851,NaN,NaN,NaN
1,Cell 17_WT_Iono_180_S,27.301812,226.0,0.925305,137.577655,2.622554,-0.3,2.931590,14.201417,229.0,0.321727,190.618241,2.682355,-0.530406,-0.532259,NaN,NaN,NaN
2,Cell 17_WT_Iono_180_S,51.795305,327.0,0.888553,116.237441,2.598708,0.0,3.726077,20.998445,327.0,0.238469,142.839469,2.605136,-0.266020,-0.205702,NaN,NaN,NaN
3,Cell 17_WT_Iono_180_S,77.488480,427.0,0.850693,125.005325,2.412892,0.5,5.256002,26.427918,422.0,0.166401,117.267368,2.324899,0.077380,0.152028,NaN,NaN,NaN
4,Cell 17_WT_Iono_180_S,92.060707,487.0,0.213380,63.638574,1.631564,0.5,3.084750,29.198897,482.0,0.062836,44.504701,1.365397,0.191339,0.279766,NaN,NaN,NaN
5,Cell 17_WT_Iono_180_S,128.180180,628.0,2.685433,132.836403,2.408838,0.4,10.658677,33.435873,624.0,0.247598,136.419872,2.587448,-0.035835,0.134562,NaN,NaN,NaN
6,Cell 17_WT_Iono_180_S,170.806902,720.0,0.356550,82.927932,2.091073,0.8,3.846753,37.652082,712.0,0.064776,38.164574,1.774659,0.447634,0.526337,NaN,NaN,NaN
7,Cell 17_WT_Iono_180_S,195.183670,796.0,2.026235,105.297397,2.487891,0.4,9.296942,40.480414,792.0,0.210882,95.772729,2.488749,0.095247,0.154952,NaN,NaN,NaN
8,Cell 17_WT_Iono_180_S,235.223953,892.0,0.393930,86.637754,2.220415,0.6,6.096185,44.809073,886.0,0.064551,59.966940,1.944192,0.266708,0.398082,NaN,NaN,NaN
9,Cell 17_WT_Iono_180_S,254.299828,986.0,1.306950,107.519738,2.440266,0.8,10.027049,46.491017,978.0,0.116595,63.028209,2.282045,0.444915,0.540933,NaN,NaN,NaN


""


,Cell ID,RCaMP Area,RCaMP Loc,RCaMP Prominence,RCaMP Start,RCaMP Width
0,Cell 17_WT_Iono_180_S,0.179583,528.0,0.035897,5158.352618,12.164738
1,Cell 17_WT_Iono_180_S,0.191659,1016.0,0.033893,10068.437988,9.156201
2,Cell 18_WT_Iono_180_S,0.264322,1495.0,0.021480,14713.602777,23.639722
3,Cell 18_WT_Iono_180_S,0.178345,684.0,0.029328,6613.228188,22.677181
4,Cell 18_WT_Iono_180_S,0.245406,237.0,0.048233,2280.118347,8.988165
5,Cell 18_WT_Iono_180_S,0.154453,1015.0,0.018523,9994.770969,15.522903
6,Cell 18_WT_Iono_180_S,0.168662,1630.0,0.031208,16051.039897,24.896010
7,Cell 18_WT_Iono_180_S,0.079195,831.0,0.015358,8087.043630,22.295637


## Inspect WT Results

In [10]:
def f(Experiment):
    data= wt_results.index[wt_results['Name']==Experiment][0]
    wt_results['Cell'][data].inspect_peaks()
interact(f, Experiment=wt_results['Name'])

interactive(children=(Dropdown(description='Experiment', options=('Cell 17_WT_Iono_180_S', 'Cell 18_WT_Iono_18…

<function __main__.f(Experiment)>

In [11]:
def f(Experiment):
    data= wt_results.index[wt_results['Name']==Experiment][0]
    wt_results['Cell'][data].inspect_results()
interact(f, Experiment=wt_results['Name'])

interactive(children=(Dropdown(description='Experiment', options=('Cell 17_WT_Iono_180_S', 'Cell 18_WT_Iono_18…

<function __main__.f(Experiment)>

## Cycle Through YAC128 Cells to Extract Peak Data


In [12]:
YAC128_Stats = pd.DataFrame()
YAC128_Shared_Peaks = pd.DataFrame()
YAC128_GCaMP = pd.DataFrame()
YAC128_RCaMP = pd.DataFrame()

YAC128_GTrace = pd.DataFrame()
YAC128_RTrace = pd.DataFrame()

YAC128_GDFF = []
YAC128_RDFF = []

YAC128_GDFF_index = []
YAC128_RDFF_index = []

for i in range(yac128_results.shape[0]):
    YAC128_GDFF_index.append(yac128_results.iloc[i,1].cell)
    YAC128_RDFF_index.append(yac128_results.iloc[i,1].cell)
    
    YAC128_RDFF.append(pd.DataFrame(yac128_results.iloc[i,1].nndR))
    YAC128_GDFF.append(pd.DataFrame(yac128_results.iloc[i,1].nndG))
    
    YAC128_RTrace = YAC128_RTrace.append(pd.Series([yac128_results.iloc[i,1].cell, yac128_results.iloc[i,1].nndR], name=yac128_results.iloc[i,1].cell))
    YAC128_GTrace = YAC128_GTrace.append(pd.Series([yac128_results.iloc[i,1].cell, yac128_results.iloc[i,1].nndG], name=yac128_results.iloc[i,1].cell)) 

    
    YAC128_Stats = YAC128_Stats.append(yac128_results.iloc[i,1].cell_stats, ignore_index=True)
    YAC128_Shared_Peaks = YAC128_Shared_Peaks.append(yac128_results.iloc[i,1].shared_peak_data, ignore_index=True)
    YAC128_GCaMP = YAC128_GCaMP.append(yac128_results.iloc[i,1].gcamp_peak_data, ignore_index=True)
    YAC128_RCaMP = YAC128_RCaMP.append(yac128_results.iloc[i,1].rcamp_peak_data, ignore_index=True)

YAC128_GDFF = pd.concat(YAC128_GDFF, axis=1) 
YAC128_GDFF = YAC128_GDFF.transpose()
YAC128_GDFF.insert(loc=0, column='Cell', value=YAC128_GDFF_index)
YAC128_RDFF = pd.concat(YAC128_RDFF, axis=1)   
YAC128_RDFF = YAC128_RDFF.transpose()
YAC128_RDFF.insert(loc=0, column='Cell', value=YAC128_RDFF_index)

## Save Results for YAC128 to CSV

In [13]:
np.set_printoptions(threshold=8000)
YAC128_Stats.to_csv("YAC128_Stats.csv", index=False)
YAC128_Shared_Peaks.to_csv("YAC128_Shared_Peaks.csv", index=False)
YAC128_GCaMP.to_csv("YAC128_GCaMP.csv", index=False)
YAC128_RCaMP.to_csv("YAC128_RCaMP.csv", index=False)

YAC128_RTrace.to_csv("YAC128_R_Trace.csv", index=False)
YAC128_GTrace.to_csv("YAC128_G_Trace.csv", index=False)
YAC128_RDFF.to_csv("YAC128_R_DFF.csv", index=False)
YAC128_GDFF.to_csv("YAC128_G_DFF.csv", index=False)

np.set_printoptions(threshold=10)

## Display Results for YAC128

In [14]:
display(YAC128_Stats)
display(YAC128_Shared_Peaks)
display(YAC128_GCaMP)
display(YAC128_RCaMP)


,Cell ID,GCaMP Peaks,RCaMP Peaks,Shared Peaks,GCaMP Percent Shared,RCaMP Percent Shared,Experiment Length (s),Drug Application,Iono GCaMP Max,Iono GCaMP Min,Iono GCaMP Dif,Iono RCaMP Max,Iono RCaMP Min,Iono RCaMP Dif,Prominence Ratio Mean,Prominence Ratio SEM
0,Cell 14_YAC128_Iono_270_S,7,10,7,1.0,0.700000,343.2,270.0,48.330062,7.238120,41.091942,48.378099,19.262397,29.115702,2.566163,0.734496
1,Cell 22_YAC128_Iono_180_D,8,12,8,1.0,0.666667,201.9,180.0,NaN,NaN,NaN,NaN,NaN,NaN,2.251150,0.841597
2,Cell 29_YAC128_Iono_180_D,9,10,9,1.0,0.900000,243.9,180.0,NaN,NaN,NaN,NaN,NaN,NaN,0.621776,0.095414
3,Cell 46_YAC128_Iono_060_S,5,9,3,0.6,0.333333,144.7,60.0,62.116394,11.115967,51.000427,36.330383,27.365784,8.964600,0.070563,0.037504
4,Cell 47_YAC128_Iono_180_S,18,18,18,1.0,1.000000,278.3,180.0,66.984497,9.069519,57.914978,38.339478,21.618530,16.720947,NaN,NaN
5,Cell 48_YAC128_Iono_088_S,9,11,9,1.0,0.818182,239.8,88.0,59.341553,9.948181,49.393372,18.984314,11.682556,7.301758,2.655380,0.440310
6,Cell 49_YAC128_Iono_155_S,1,2,1,1.0,0.500000,340.5,155.0,59.104187,9.760925,49.343262,45.403992,28.798828,16.605164,0.978201,0.978201


,Cell ID,GCaMP Area,GCaMP Loc (Frame),GCaMP Prominence,GCaMP Start,GCaMP Width,Peak Time Diff (G-R),Promicence Ratio (G/R),RCaMP Area,RCaMP Loc (Frame),RCaMP Prominence,RCaMP Start,RCaMP Width,Start Difference (G-R) Seconds,Start Difference 10% Max(G-R) Seconds,GCaMP Loc,RCaMP Loc,Start Difference (G-R)
0,Cell 14_YAC128_Iono_270_S,2.002919,184.0,0.256778,75.012095,1.767141,0.5,0.483436,3.2102,179.0,0.484233,71.913218,1.807383,0.030989,0.272846,NaN,NaN,NaN
1,Cell 14_YAC128_Iono_270_S,16.351654,427.0,1.303250,88.867780,1.991547,0.8,1.649555,14.8439,419.0,0.635266,76.973586,1.941712,0.118942,0.380387,NaN,NaN,NaN
2,Cell 14_YAC128_Iono_270_S,71.772137,815.0,3.363962,119.283951,2.205480,0.3,4.748087,39.0152,812.0,0.870929,153.317224,2.282126,-0.340333,-0.421153,NaN,NaN,NaN
3,Cell 14_YAC128_Iono_270_S,149.345688,1554.0,3.667970,88.553566,2.406652,-0.5,4.066429,64.4151,1559.0,0.926323,166.408947,2.558432,-0.778554,-0.858916,NaN,NaN,NaN
4,Cell 14_YAC128_Iono_270_S,246.116822,1792.0,3.595802,84.266814,2.411606,-0.2,4.545052,85.6541,1794.0,0.794040,128.950859,2.490309,-0.446840,-0.554770,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Cell 48_YAC128_Iono_088_S,208.673392,800.0,0.689775,107.171602,2.330350,0.8,4.400700,48.9246,792.0,0.148089,65.773952,2.080386,0.413976,0.565044,NaN,NaN,NaN
68,Cell 48_YAC128_Iono_088_S,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.380621,NaN,0.060082,219.525337,20.247466,NaN,NaN,NaN,422.0,NaN
69,Cell 48_YAC128_Iono_088_S,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.298346,NaN,0.053020,435.504025,16.349598,NaN,NaN,NaN,599.0,NaN
70,Cell 49_YAC128_Iono_155_S,17.218359,198.0,1.847238,97.920007,2.062700,0.7,1.956403,8.17438,191.0,0.866810,101.234551,2.181728,-0.033145,0.193372,NaN,NaN,NaN


,Cell ID,GCaMP Area,GCaMP Loc,GCaMP Prominence,GCaMP Start,GCaMP Width
0,Cell 46_YAC128_Iono_060_S,0.017919,442.0,0.003333,310.421281,13.157872
1,Cell 46_YAC128_Iono_060_S,0.002201,211.0,0.001529,137.603829,7.339617


,Cell ID,RCaMP Area,RCaMP Loc,RCaMP Prominence,RCaMP Start,RCaMP Width
0,Cell 14_YAC128_Iono_270_S,0.433526,2497.0,0.083295,24779.977059,19.002294
1,Cell 14_YAC128_Iono_270_S,1.008035,1837.0,0.210385,18263.932462,10.606754
2,Cell 14_YAC128_Iono_270_S,0.516322,625.0,0.119352,6071.220096,17.877990
3,Cell 22_YAC128_Iono_180_D,0.595342,1225.0,0.084559,12053.589374,19.641063
4,Cell 22_YAC128_Iono_180_D,0.764323,898.0,0.098654,8752.863134,22.713687
5,Cell 22_YAC128_Iono_180_D,0.445754,1676.0,0.038109,16534.063218,22.593678
6,Cell 22_YAC128_Iono_180_D,0.253517,1071.0,0.034356,10494.728235,21.527176
7,Cell 29_YAC128_Iono_180_D,0.364294,759.0,0.045443,7372.690192,21.730981
8,Cell 46_YAC128_Iono_060_S,0.003828,544.0,0.001605,5349.823748,9.017625
9,Cell 46_YAC128_Iono_060_S,0.053016,460.0,0.007431,4482.139397,11.786060


## Inspect YAC128 Results

In [15]:
def f(Experiment):
    data= yac128_results.index[yac128_results['Name']==Experiment][0]
    yac128_results['Cell'][data].inspect_peaks()
interact(f, Experiment=yac128_results['Name'])

interactive(children=(Dropdown(description='Experiment', options=('Cell 14_YAC128_Iono_270_S', 'Cell 22_YAC128…

<function __main__.f(Experiment)>

In [16]:
def f(Experiment):
    data= yac128_results.index[yac128_results['Name']==Experiment][0]
    yac128_results['Cell'][data].inspect_results()
interact(f, Experiment=yac128_results['Name'])

interactive(children=(Dropdown(description='Experiment', options=('Cell 14_YAC128_Iono_270_S', 'Cell 22_YAC128…

<function __main__.f(Experiment)>